In [ ]:
using Base.Threads, ForceSpectroscopyMLE, Plots

println(string("Number of available cores for threading: ", nthreads()))

# Introduction

This is a minimal example on how to apply our framework to a set of rupture forces measured at different pulling velocities.  We assume that the data files are stored in the directory `rupture_forces`.  Each file should be of the form:

In [ ]:
example_F = [ 129.0, 134.0, 137.0, 130.0 ] # pN
example_dF = [ 10.0, 15.5, 10.8, 10.3, 8.2 ] # pN/s
example_data = hcat([example_F, example_dF]...)

This is a minimal example on how to apply our framework to a set of rupture forces measured at different pulling velocities.  We assume that the data files (consisting of columns of rupture forces and associated loading rates, in that order) are stored in the directory `rupture_forces`. Reading in the data goes as follows:

In [ ]:
data = read_data("./rupture_forces/")
all_data = vcat(data...);

The analysis depends on the choice of the parameter $\nu \in \{1/2, 2/3, 1 \}$, which we fix below:

In [ ]:
const ν = 2/3;

For more details on the theoretical framework, please refer to the associated publication:

> W. Cai, J. T. Bullerjahn, M. Lallemang, K. Kroy, B. N. Balzer, and T. Hugel, "Direction dependence of bond strength and polymer chain elasticity", submitted.

# Data trimming

We now systematically trim the $i = 0, 1, 2, \dots, i_{\text{max}}$ largest rupture forces from each subsample `data[k]` of our data to explore how that effects the parameter estimates.  We then compare the results to the case, where $i$ randomly chosen rupture forces are excluded.  

In [ ]:
const i_max = 10
params = data_exclusion([data],ν,i_max)
random_params = random_data_exclusion([data],ν,i_max);

In [ ]:
p = plot(params[1,:],hcat([params[2,:],random_params[2,:]]...), 
    legend=:topleft, labels=["trimming from above" "random exclusion of points"], 
    xlabel="percentage of excluded data points", ylabel="βΔG_u")

In [ ]:
p = plot(params[1,:],hcat([params[3,:],random_params[3,:]]...), 
    legend=:topleft, labels=["trimming from above" "random exclusion of points"], 
    xlabel="percentage of excluded data points", ylabel="x_u [nm]")

In [ ]:
p = plot(params[1,:], hcat([params[4,:],random_params[4,:]]...), 
    yscale=:log10, yticks=[ 10.0^(-i) for i = 1 : 3], yminorticks=true, 
    legend=:bottomleft, labels=["trimming from above" "random exclusion of points"], 
    xlabel="percentage of excluded data points", ylabel="k_0 [1/s]")

The resulting mean rupture force $\langle F \rangle$ converges with increasing $i$, as demonstrated below:

In [ ]:
const β = 1/(1.38064852e-23*1e12*1e9*295) # Temperature assumed at 

Fdot = [ exp(x) for x in round(log(10^3);digits=1):0.1:round(log(2*10^6);digits=1) ];
mF_0 = meanF.(β,params[2,0+1],params[3,0+1],params[4,0+1],2/3,Fdot)
mF_1 = meanF.(β,params[2,1+1],params[3,1+1],params[4,1+1],2/3,Fdot)
mF_2 = meanF.(β,params[2,2+1],params[3,2+1],params[4,2+1],2/3,Fdot)
mF_3 = meanF.(β,params[2,3+1],params[3,3+1],params[4,3+1],2/3,Fdot)
mF_4 = meanF.(β,params[2,4+1],params[3,4+1],params[4,4+1],2/3,Fdot)
mF_5 = meanF.(β,params[2,5+1],params[3,5+1],params[4,5+1],2/3,Fdot)
#sdF = sqrt.(varF.(params[2,i+1],params[3,i+1],params[4,i+1],2/3,Fdot))

plot(all_data[:,2], all_data[:,1], seriestype = :scatter, color=:lightgrey, 
    xscale=:log10, xlims=(2*10^3,1.5*10^6), xticks=[ 10^i for i = 1 : 6], xminorticks=true, 
    ylims=(0.0,3000.0), 
    legend=:topleft, label="experimental data", 
    xlabel="loading rate [pN/s]", 
    ylabel="rupture force [pN]")
p = plot!(Fdot, hcat([mF_0, mF_1, mF_3, mF_5]...), #=ribbon=sdF,=# 
    labels=["i=0" "i=1" "i=3" "i=5"], linewidth=2)

From the plots above, we read off the threshold value $i_{\text{trim}}$, where the slope of the parameter curves suddenly switches.  The resulting trimmed data can then be analyzed to extract parameter and error estimates.  

# MLE analysis of the data

In [ ]:
const i_trim = 5 # determined from the above plots
trimmed_data = reduce_data(data,i_trim)
trimmed_all_data = vcat(trimmed_data...);

In [ ]:
const N = 100 # number of resampled data sets for bootstrapping
global_estimates = MLE_estimator(trimmed_all_data,ν)
global_errors = MLE_errors(trimmed_all_data,ν,N)

println("Estimates:")
println(string("βΔG_u = ", global_estimates[1], " ± ", global_errors[1]))
println(string("x_u = ", global_estimates[2], " ± ", global_errors[2]))
println(string("k_0 = ", global_estimates[3], " ± ", global_errors[3]))

Note that the error estimates can vary significantly between runs if the number $N$ of bootstrapping samples is small.  For large $N$ the computation can become sluggish, so it is recommended to run the command `export JULIA_NUM_THREADS=`, with `n` being the number of available (physical) cores, before launching Julia.  This speeds up the numerics significantly.  